## EDA for Content Based Recommendation System
- [Source 1](https://github.com/ploomber/sql/blob/main/mini-projects/movie-rec-system/movie_rec_system/etl/eda.ipynb)
- [Source 2](https://ploomber-sql.readthedocs.io/en/latest/mini-projects/recommendation-system/eda-with-jupyter.html)

In [ ]:
%reload_ext sql
%sql duckdb:///../movies_data.duckdb


In [ ]:
%sqlcmd columns -t movies

In [ ]:
%sqlcmd columns -t genres

In [ ]:
%%sql
SELECT *
FROM movies
LIMIT 3

In [ ]:
%%sql
SELECT *
FROM genres
LIMIT 2

## Expanding genre name from id

In [ ]:
%%sql
WITH ExpandedGenres AS (
    SELECT
        m.id AS movie_id,
        mg.movie_genre_id,
        g.name AS genre_name
    FROM
        (SELECT UNNEST(movies.genre_ids) as movie_genre_id, movies.id FROM movies) AS mg
    JOIN
        movies m ON mg.id = m.id
    JOIN
        genres g ON mg.movie_genre_id = g.id
)

SELECT
    movie_id,
    STRING_AGG(genre_name, ', ') AS genre_names
FROM
    ExpandedGenres
GROUP BY
    movie_id;

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS movie_genre_data AS
WITH ExpandedGenres AS (
    SELECT
        m.id AS movie_id,
        mg.movie_genre_id,
        g.name AS genre_name
    FROM
        (SELECT UNNEST(movies.genre_ids) as movie_genre_id, movies.id FROM movies) AS mg
    JOIN
        movies m ON mg.id = m.id
    JOIN
        genres g ON mg.movie_genre_id = g.id
),
genre_names AS (
    SELECT
    movie_id,
    STRING_AGG(genre_name, ', ') AS genre_names
FROM
    ExpandedGenres
GROUP BY
    movie_id
)
SELECT gn.genre_names, m.id, m.original_language,
       m.overview, m.popularity, m.release_date,
       m.title, m.vote_average, m.vote_count
FROM genre_names gn
JOIN movies m
ON gn.movie_id = m.id
WHERE m.vote_count != 0

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
df = %sql SELECT * FROM movie_genre_data
df = df.DataFrame()
df.head(2)


In [ ]:
ax = %sqlplot histogram --table movie_genre_data --column popularity --bins 20
ax.grid()
ax.set_title("Movie Popularity")
ax.set_xlabel("Popularity")


In [ ]:
ax = %sqlplot histogram --table movie_genre_data --column vote_average --bins 20
ax.grid()
ax.set_title("Vote Average")
ax.set_xlabel("Vote Score")


In [ ]:
ax = %sqlplot histogram --table movie_genre_data --column vote_count --bins 20
ax.grid()
ax.set_title("Movie Vote Count")
ax.set_xlabel("Votes")


In [ ]:
df["release_date"] = pd.to_datetime(df["release_date"])

df["release_year"] = df["release_date"].dt.year

movie_counts_by_year = df.groupby("release_year").size()

plt.figure(figsize=(10, 6))
plt.plot(movie_counts_by_year.index, movie_counts_by_year.values, marker="o")
plt.xlabel("Release Year")
plt.ylabel("Number of Movies Released")
plt.title("Number of Movies Released Per Year")
plt.tight_layout()
plt.grid()
plt.show()


In [ ]:
def get_genre_count(df):
    genre_counts = dict()

    for genres in df["genre_names"]:
        genre_list = genres.split(",")
        for genre in genre_list:
            if genre not in genre_counts:
                genre_counts[genre] = 1
            genre_counts[genre] += 1
    return genre_counts


In [ ]:
df["genre_names"] = df["genre_names"].astype(str)

In [ ]:
genre_counts = get_genre_count(df)

sorted_genre_counts = dict(
    sorted(genre_counts.items(), key=lambda item: item[1], reverse=True)
)

genres = list(sorted_genre_counts.keys())
counts = list(sorted_genre_counts.values())

plt.figure(figsize=(16, 7))
plt.bar(genres, counts)
plt.ylabel("Genres")
plt.xlabel("Counts")
plt.title("Genre Counts")
plt.tight_layout()
plt.xticks(rotation=60, ha="right")
plt.grid()
plt.show()
